In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
data = pd.read_excel('个股收盘价收益率成交量.xlsx', encoding='gbk', usecols=[0, 1, 5], parse_dates=[0]).sort_values(by=['股票代码', '日期'])

In [3]:
index_data = pd.read_csv('上证50收益率.csv', encoding='gbk', parse_dates=[0], usecols=[0, 3], skiprows=1, names=['日期', '上证50收益率'])[::-1]

In [4]:
data = data.merge(index_data, on='日期')

In [5]:
data = data.sort_values(['股票代码', '日期'])

In [6]:
data.reset_index(drop=True, inplace=True)

In [7]:
data['超额收益率'] = data['回报率(不考虑分红)'] - data['上证50收益率']

In [8]:
data['ret20'] = data.groupby('股票代码')['超额收益率'].rolling(20).sum().values

In [9]:
data['ret60'] = data.groupby('股票代码')['超额收益率'].rolling(60).sum().values

In [10]:
data['volitility'] = data.groupby('股票代码')['回报率(不考虑分红)'].rolling(20).std().values

In [11]:
size_volume = pd.read_csv('市值、成交量.csv', usecols=[0, 1, 3, 4], parse_dates=[0], skiprows=1, names=['日期', '股票代码', '市值', '成交量'])

In [12]:
data = data.merge(size_volume, on=['股票代码', '日期'])

In [13]:
net_asset = pd.read_csv('所有者权益(1).csv', encoding='utf8')

In [14]:
net_asset['report_quarter'] = net_asset['report_quarter'] % 1000 * 3

In [15]:
net_asset['report_day'] = 30

In [16]:
net_asset.rename(columns={'report_year': 'year', 'report_quarter': 'month', 'report_day': 'day'}, inplace=True)

In [17]:
net_asset.index = pd.to_datetime(net_asset[['year', 'month', 'day']])

In [18]:
net_asset_daily = pd.DataFrame()
tickers = pd.unique(net_asset['stock_code'])

In [19]:
for ticker in tickers:
    tk_df = net_asset[net_asset['stock_code'] == ticker]
    tk_df = tk_df.resample('D').ffill()
    net_asset_daily = pd.concat([net_asset_daily, tk_df], axis=0)

In [20]:
net_asset_daily = net_asset_daily.iloc[:, [0, 1, 4]]

In [21]:
net_asset_daily.reset_index(inplace=True)

In [22]:
net_asset_daily.rename(columns={'index': '日期', 'stock_code': '股票代码'}, inplace=True)

In [23]:
data = data.merge(net_asset_daily, on=['日期', '股票代码'])

In [24]:
data['账面市值比'] = data['tse'] / data['市值']

In [25]:
con_roe = pd.read_csv('一致预期净资产收益率.csv', skiprows=1, usecols=[0, 1, 3], names=['日期', '股票代码', 'con_roe'], parse_dates=[0])

In [26]:
data = data.merge(con_roe, on=['日期', '股票代码'])

In [27]:
con_rating_strength = pd.read_csv('一致预期评级强度.csv', usecols=[0, 1, 3], skiprows=1, names=['日期', '股票代码', 'con_rating_strength'], parse_dates=[0])

In [28]:
data = data.merge(con_rating_strength, on=['日期', '股票代码'])

In [29]:
report_num_m = pd.read_csv('个股月报告数.csv', usecols=[0, 1, 3], skiprows=1, names=['日期', '股票代码', 'report_num_m'], parse_dates=[0])
data = data.merge(report_num_m, on=['日期', '股票代码'])

In [30]:
relative_report_num_10 = pd.read_csv('个股10日相对报告数.csv', usecols=[0, 1, 3], skiprows=1, names=['日期', '股票代码', 'relative_report_num_10'], parse_dates=[0])
data = data.merge(relative_report_num_10, on=['日期', '股票代码'])

In [31]:
con_eps_roll = pd.read_csv('滚动一致预期每股收益.csv', skiprows=1, names=['股票代码', '日期', 'con_eps_roll'], parse_dates=[1])
data = data.merge(con_eps_roll, on=['日期', '股票代码'])

In [32]:
eps = pd.read_csv('基本每股收益.csv', encoding='utf8')
eps['report_quarter'] = eps['report_quarter'] % 1000 * 3
eps['report_day'] = 30
eps.rename(columns={'report_year': 'year', 'report_quarter': 'month', 'report_day': 'day'}, inplace=True)
eps.index = pd.to_datetime(eps[['year', 'month', 'day']])
eps_daily = pd.DataFrame()
for ticker in tickers:
    tk_df = eps[eps['stock_code'] == ticker]
    tk_df = tk_df.resample('D').ffill()
    eps_daily = pd.concat([eps_daily, tk_df], axis=0)
eps_daily = eps_daily.iloc[:, [0, 3]]
eps_daily.reset_index(inplace=True)
eps_daily.rename(columns={'index': '日期', 'stock_code': '股票代码'}, inplace=True)
data = data.merge(eps_daily, on=['日期', '股票代码'])

In [33]:
data['Exc_eps'] = data['con_eps_roll'] - data['bas_eps']

In [34]:
data['Exc20'] = data.groupby('股票代码')['Exc_eps'].rolling(20).mean().values
data['Exc60'] = data.groupby('股票代码')['Exc_eps'].rolling(60).mean().values

In [35]:
end_date = pd.to_datetime(20200123, format='%Y%m%d')

In [36]:
end_date

Timestamp('2020-01-23 00:00:00')

In [37]:
senti_path = 'all_result'
senti_file_list = os.listdir(senti_path)
senti_data = pd.DataFrame()
for file in senti_file_list:
    senti = pd.read_csv(os.path.join(senti_path, file), usecols=[0] + list(range(9, 17)), parse_dates=[0], encoding='gbk')[::-1]
    senti.rename(columns={'post_date': '日期'}, inplace=True)
    senti = senti[senti['日期'] <= end_date]
    senti_count = senti.groupby('日期')['情感'].value_counts().unstack().fillna(0)
    senti_count1 = senti.groupby('日期')['主题一'].value_counts().unstack().fillna(0)
    senti_count2 = senti.groupby('日期')['主题二'].value_counts().unstack().fillna(0)
    senti_count3 = senti.groupby('日期')['主题三'].value_counts().unstack().fillna(0)
    senti_count4 = senti.groupby('日期')['主题四'].value_counts().unstack().fillna(0)
    senti_count5 = senti.groupby('日期')['主题五'].value_counts().unstack().fillna(0)
    senti_count6 = senti.groupby('日期')['主题六'].value_counts().unstack().fillna(0)
    senti_count7 = senti.groupby('日期')['主题七'].value_counts().unstack().fillna(0)
    senti_count = senti_count.rolling(20).sum()
    senti_count1 = senti_count1.rolling(20).sum()
    senti_count2 = senti_count2.rolling(20).sum()
    senti_count3 = senti_count3.rolling(20).sum()
    senti_count4 = senti_count4.rolling(20).sum()
    senti_count5 = senti_count5.rolling(20).sum()
    senti_count6 = senti_count6.rolling(20).sum()
    senti_count7 = senti_count7.rolling(20).sum()
    senti_count['Opin1'] = np.log((1 + senti_count['积极']) / (1 + senti_count['消极']))
    senti_count['Opin2'] = (senti_count['积极'] - senti_count['消极']) / (senti_count['积极'] + senti_count['消极'] + senti_count['中性'])
    senti_count = senti_count[['Opin1', 'Opin2']]
    
    senti_count1['主题一Opin1'] = np.log((1 + senti_count1['积极']) / (1 + senti_count1['消极']))
    senti_count1['主题一Opin2'] = (senti_count1['积极'] - senti_count1['消极']) / (senti_count1['积极'] + senti_count1['消极'] + senti_count1['中性'])
    senti_count1['主题一pos'] = senti_count1['积极'] / (senti_count1['积极'] + senti_count1['消极'] + senti_count1['中性'])
    senti_count1['主题一neg'] = senti_count1['消极'] / (senti_count1['积极'] + senti_count1['消极'] + senti_count1['中性'])
    senti_count1 = senti_count1[['主题一Opin1', '主题一Opin2', '主题一pos', '主题一neg']]
    
    senti_count2['主题二Opin1'] = np.log((1 + senti_count2['积极']) / (1 + senti_count2['消极']))
    senti_count2['主题二Opin2'] = (senti_count2['积极'] - senti_count2['消极']) / (senti_count2['积极'] + senti_count2['消极'] + senti_count2['中性'])
    senti_count2['主题二pos'] = senti_count2['积极'] / (senti_count2['积极'] + senti_count2['消极'] + senti_count2['中性'])
    senti_count2['主题二neg'] = senti_count2['消极'] / (senti_count2['积极'] + senti_count2['消极'] + senti_count2['中性'])
    senti_count2 = senti_count2[['主题二Opin1', '主题二Opin2', '主题二pos', '主题二neg']]
    
    senti_count3['主题三Opin1'] = np.log((1 + senti_count3['积极']) / (1 + senti_count3['消极']))
    senti_count3['主题三Opin2'] = (senti_count3['积极'] - senti_count3['消极']) / (senti_count3['积极'] + senti_count3['消极'] + senti_count3['中性'])
    senti_count3['主题三pos'] = senti_count3['积极'] / (senti_count3['积极'] + senti_count3['消极'] + senti_count3['中性'])
    senti_count3['主题三neg'] = senti_count3['消极'] / (senti_count3['积极'] + senti_count3['消极'] + senti_count3['中性'])
    senti_count3 = senti_count3[['主题三Opin1', '主题三Opin2', '主题三pos', '主题三neg']]
    
    senti_count4['主题四Opin1'] = np.log((1 + senti_count4['积极']) / (1 + senti_count4['消极']))
    senti_count4['主题四Opin2'] = (senti_count4['积极'] - senti_count4['消极']) / (senti_count4['积极'] + senti_count4['消极'] + senti_count4['中性'])
    senti_count4['主题四pos'] = senti_count4['积极'] / (senti_count4['积极'] + senti_count4['消极'] + senti_count4['中性'])
    senti_count4['主题四neg'] = senti_count4['消极'] / (senti_count4['积极'] + senti_count4['消极'] + senti_count4['中性'])
    senti_count4 = senti_count4[['主题四Opin1', '主题四Opin2', '主题四pos', '主题四neg']]
    
    senti_count5['主题五Opin1'] = np.log((1 + senti_count5['积极']) / (1 + senti_count5['消极']))
    senti_count5['主题五Opin2'] = (senti_count5['积极'] - senti_count5['消极']) / (senti_count5['积极'] + senti_count5['消极'] + senti_count5['中性'])
    senti_count5['主题五pos'] = senti_count5['积极'] / (senti_count5['积极'] + senti_count5['消极'] + senti_count5['中性'])
    senti_count5['主题五neg'] = senti_count5['消极'] / (senti_count5['积极'] + senti_count5['消极'] + senti_count5['中性'])
    senti_count5 = senti_count5[['主题五Opin1', '主题五Opin2', '主题五pos', '主题五neg']]
    
    senti_count6['主题六Opin1'] = np.log((1 + senti_count6['积极']) / (1 + senti_count6['消极']))
    senti_count6['主题六Opin2'] = (senti_count6['积极'] - senti_count6['消极']) / (senti_count6['积极'] + senti_count6['消极'] + senti_count6['中性'])
    senti_count6['主题六pos'] = senti_count6['积极'] / (senti_count6['积极'] + senti_count6['消极'] + senti_count6['中性'])
    senti_count6['主题六neg'] = senti_count6['消极'] / (senti_count6['积极'] + senti_count6['消极'] + senti_count6['中性'])
    senti_count6 = senti_count6[['主题六Opin1', '主题六Opin2', '主题六pos', '主题六neg']]
    
    senti_count7['主题七Opin1'] = np.log((1 + senti_count7['积极']) / (1 + senti_count7['消极']))
    senti_count7['主题七Opin2'] = (senti_count7['积极'] - senti_count7['消极']) / (senti_count7['积极'] + senti_count7['消极'] + senti_count7['中性'])
    senti_count7['主题七pos'] = senti_count7['积极'] / (senti_count7['积极'] + senti_count7['消极'] + senti_count7['中性'])
    senti_count7['主题七neg'] = senti_count7['消极'] / (senti_count7['积极'] + senti_count7['消极'] + senti_count7['中性'])
    senti_count7 = senti_count7[['主题七Opin1', '主题七Opin2', '主题七pos', '主题七neg']]
    
    senti = pd.concat([senti_count, senti_count1, senti_count2, senti_count3, senti_count4, senti_count5, senti_count6, senti_count7], axis=1)
    senti['股票代码'] = int(file.split('.')[0])
    senti.reset_index(inplace=True)
    senti_data = pd.concat([senti_data, senti])

In [38]:
senti_data

,日期,Opin1,Opin2,主题一Opin1,主题一Opin2,主题一pos,主题一neg,主题二Opin1,主题二Opin2,主题二pos,...,主题五neg,主题六Opin1,主题六Opin2,主题六pos,主题六neg,主题七Opin1,主题七Opin2,主题七pos,主题七neg,股票代码
0,2014-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600016
1,2014-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600016
2,2014-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600016
3,2014-03-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600016
4,2014-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2016-05-16,-0.204794,-0.085470,-0.080043,-0.024390,0.268293,0.292683,0.000000,0.000000,0.457627,...,NaN,-0.747214,-0.294118,0.235294,0.529412,NaN,NaN,NaN,NaN,601336
1530,2016-05-17,-0.209092,-0.087719,NaN,NaN,NaN,NaN,0.000000,0.000000,0.457627,...,NaN,-0.693147,-0.277778,0.250000,0.527778,NaN,NaN,NaN,NaN,601336
1531,2016-05-18,-0.206336,-0.085271,-0.154151,-0.045455,0.250000,0.295455,0.000000,0.000000,0.459016,...,NaN,-0.559616,-0.225000,0.275000,0.500000,-0.597837,-0.257143,0.285714,0.542857,601336
1532,2016-05-19,-0.271934,-0.111940,NaN,NaN,NaN,NaN,0.138150,0.066667,0.500000,...,NaN,-0.646627,-0.256410,0.256410,0.512821,NaN,NaN,NaN,NaN,601336


In [39]:
data = senti_data.merge(data, on=['股票代码', '日期'])

In [40]:
data.set_index(['股票代码', '日期'], inplace=True)

In [41]:
data['const'] = 1

In [42]:
data['市值'] = np.log(data['市值'])
data['成交量'] = np.log(data['成交量'])

D:\Anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [43]:
data.dropna(inplace=True)

In [44]:
data['成交量'] = data['成交量'].groupby('股票代码').transform(lambda x: (x - x.mean()) / x.std())

In [46]:
data.columns

Index(['Opin1', 'Opin2', '主题一Opin1', '主题一Opin2', '主题一pos', '主题一neg',
       '主题二Opin1', '主题二Opin2', '主题二pos', '主题二neg', '主题三Opin1', '主题三Opin2',
       '主题三pos', '主题三neg', '主题四Opin1', '主题四Opin2', '主题四pos', '主题四neg',
       '主题五Opin1', '主题五Opin2', '主题五pos', '主题五neg', '主题六Opin1', '主题六Opin2',
       '主题六pos', '主题六neg', '主题七Opin1', '主题七Opin2', '主题七pos', '主题七neg',
       '回报率(不考虑分红)', '上证50收益率', '超额收益率', 'ret20', 'ret60', 'volitility', '市值',
       '成交量', 'stock_name', 'tse', '账面市值比', 'con_roe', 'con_rating_strength',
       'report_num_m', 'relative_report_num_10', 'con_eps_roll', 'bas_eps',
       'Exc_eps', 'Exc20', 'Exc60', 'const'],
      dtype='object')

In [48]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm

In [55]:
exog_vars = ['Opin1', 'Opin2', 'volitility', '市值', '账面市值比', '成交量', 'report_num_m', 'relative_report_num_10', 'con_rating_strength', 'con_roe', 'const']
exog = data[exog_vars]
mod = PanelOLS(data['ret60'], exog, entity_effects=True, time_effects=True)
result = mod.fit()
with open('回归结果/未分类三个月超额收益.txt', 'w') as f:
    f.write(str(result))
    f.close()

3099

In [56]:
result.summary

Dep. Variable:,ret60,R-squared:,0.2052
Estimator:,PanelOLS,R-squared (Between):,-9.5527
No. Observations:,1269,R-squared (Within):,-0.0719
Date:,"Tue, May 19 2020",R-squared (Overall):,-0.5033
Time:,20:38:08,Log-likelihood,-2964.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,9.2963
Entities:,6,P-value,0.0000
Avg Obs:,211.50,Distribution:,"F(10,360)"
Min Obs:,44.000,,
Max Obs:,509.00,F-statistic (robust):,9.2963


In [ ]:
result.params

In [ ]:
exog_vars = ['主题一Opin2', '主题二Opin2', '主题三Opin2', '市值', 'const']
exog = data[exog_vars]
mod = PanelOLS(data['成交量'], exog, entity_effects=True, time_effects=True)
result = mod.fit()
with open('回归结果/成交量——Opin2主题分类回归结果.txt', 'w') as f:
    f.write(str(result))
    f.close()

In [ ]:
exog_vars = ['Opin1', '市值', 'const']
exog = data[exog_vars]
mod = PanelOLS(data['成交量'], exog, entity_effects=True, time_effects=True)
result = mod.fit()
with open('回归结果/成交量——Opin1未经主题分类回归结果.txt', 'w') as f:
    f.write(str(result))
    f.close()

In [ ]:
exog_vars = ['Opin2', '市值', 'const']
exog = data[exog_vars]
mod = PanelOLS(data['成交量'], exog, entity_effects=True, time_effects=True)
result = mod.fit()
with open('回归结果/成交量——Opin2未经主题分类回归结果.txt', 'w') as f:
    f.write(str(result))
    f.close()

In [ ]:
result.params